<a href="https://colab.research.google.com/github/brkuhgk/yolo/blob/master/YoloVedioCap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
from google.colab.patches import cv2_imshow

In [21]:
import cv2
import numpy as np


In [22]:
#drive/MyDrive/new/yolo/

# Load Yolo 
#drive/MyDrive/new/yolo/
net = cv2.dnn.readNet("drive/MyDrive/new/yolo/yolov3.weights", "drive/MyDrive/new/yolo/yolov3.cfg")
classes = []
with open("drive/MyDrive/new/yolo/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [23]:
#vediocapture
vediocap = cv2.VideoCapture('drive/MyDrive/new/yolo/tv_chair.mp4')
vediocap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
vediocap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
vediocap.set(cv2.CAP_PROP_FPS, 30) 

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('drive/MyDrive/new/yolo/tv_chairRes1.avi',fourcc, 30.0, (640,480))


False

In [ ]:
while vediocap.isOpened():
    # Capture frame-by-frame
  ret, img = vediocap.read()
  if not ret:
    break
  # Loading image
  # img = cv2.imread("drive/MyDrive/new/yolo/")
  # img = cv2.resize(img, None, fx=0.4, fy=0.4)
  height,width,channels = img.shape
  
  #Detecting objects
  blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
  net.setInput(blob)
  outs = net.forward(output_layers)

  # Showing informations on the screen
  class_ids = []
  confidences = []
  boxes = []
  for out in outs:
      for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
  
  
  
  indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

  font = cv2.FONT_HERSHEY_PLAIN
  for i in range(len(boxes)):
      # print(i)
      if i in indexes:
          x, y, w, h = boxes[i]
          label = str(classes[class_ids[i]])
          color = colors[i]
          cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
          cv2.putText(img, label, (x, y + 30), font, 3, color, 3)

  # cv2_imshow(img)
  out.write(img)

cv2.waitKey(0)
cv2.destroyAllWindows()
